# Import Libraries

In [1]:
import random

SEED = 32
random.seed(SEED)

import numpy as np 
import pandas as pd
import spacy

from sklearn.metrics import accuracy_score
from sklearn.metrics import  f1_score

from torch import nn
import torch
from torchtext import data
from torch.nn  import functional as F
import torch.optim as  optim 
if torch.cuda.is_available():  
  dev = "cuda:0" 
  print("gpu up")
else:  
  dev = "cpu"  
device = torch.device(dev)

gpu up


In [2]:
"""

those are the libraries I use for processing text

"""

import nltk
nltk.download("punkt")

import re
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()

tokenizer = Tokenizer(nlp.vocab)

from nltk import word_tokenize,sent_tokenize
from nltk.stem  import PorterStemmer


from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

nltk.download('stopwords')
stops = stopwords.words("english")


def removepunc(my_str): # function to remove punctuation
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))
snowstem = SnowballStemmer("english")
portstem = PorterStemmer()


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Load Dataset

In [3]:
traindata = pd.read_csv("/kaggle/input/phishing-detection/train.csv")
test = pd.read_csv("/kaggle/input/phishing-detection/test.csv")
traindata.drop("Unnamed: 0",axis=1,inplace=True)
test.drop("Unnamed: 0",axis=1,inplace=True)

In [4]:
traindata

,URL,Label
0,xml.coverpages.org/xmlForms.html,2
1,pawsoft.com/files/,1
2,ibegin.com/directory/ca/quebec/anjou/,2
3,mxp4016.com,1
4,hooksgems.blogspot.com/2009/09/clark-terry-cla...,2
...,...,...
368056,pt-tkbi.com/providernet/provider/provider/webm...,1
368057,wiki.d-addicts.com/Ninomiya_Kazunari,2
368058,jlkc.org/,2
368059,picobong.com/www.redirect.com.htm,1


In [5]:
"""
this function is the tokenizer we are using, it does basic processing also  like ,
Lowercase the text
removing punctuation, stop words and numbers,
it also removes extra spaces and unwanted characters (I use regex for that)


before using the tokenizer I was testing it on the train dataframe manually  
"""

def myTokenizer(x):
 return  [snowstem.stem(word.text)for word in 
          tokenizer(removepunc(re.sub(r"\s+\s+"," ",re.sub(r"[^A-Za-z0-9()!?\'\`\"\r+\n+]"," ",x.lower()))).strip()) 
          if (word.text not in stops and not hasNumbers(word.text)) ]


In [6]:
"""
here I'm using the torchtext fields and dataset classes they can ease the work to get
the dataset ready for the pytorch model

the class DataFrameDataset is the easiest way I found to turn a dataframe into a torchtext dataset

this cell will take sometime to finish
"""

TEXT = data.Field(tokenize=myTokenizer,batch_first=True,fix_length=140)
LABEL = data.LabelField(dtype=torch.float ,batch_first=True)


class DataFrameDataset(data.Dataset):

    def __init__(self, df, text_field, label_field, is_test=False, **kwargs):
        fields = [('comment_text', text_field), ('toxic', label_field)]
        examples = []
        for i, row in df.iterrows():
            label = row.Label 
            text = row.URL
            examples.append(data.Example.fromlist([text, label], fields))

        super().__init__(examples, fields, **kwargs)
  

torchdataset = DataFrameDataset(traindata, TEXT,LABEL)
torchtest = DataFrameDataset(test, TEXT,LABEL)

# Split Dataset for train and test

In [7]:
train_data, valid_data = torchdataset.split(split_ratio=0.8, random_state = random.seed(SEED))

In [8]:
"""
this cell build the vocab which means it get all the used words and if also ignores any word 
that only appeared less than 3 times
"""
TEXT.build_vocab(train_data,min_freq=3)  
LABEL.build_vocab(train_data)


In [9]:
#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  


Size of TEXT vocabulary: 36857
Size of LABEL vocabulary: 2
[('com', 210421), ('html', 43390), ('www', 28719), ('org', 26847), ('net', 18407), ('amp', 17995), ('htm', 17088), ('php', 16645), ('en', 16334), ('index', 15289)]


In [10]:
#set batch size
BATCH_SIZE = 128

"""
we are using batches for validation and test set because of memory usage we can't pass the whole set at once 
"""


train_iterator,valid_iterator,test_iterator= data.BucketIterator.splits(
    (train_data,valid_data,torchtest), 
    batch_size = BATCH_SIZE,
    device = device,
    sort =False,
shuffle=False)


# Writing Algorithm

In [11]:

"""
one major point here is that I encoded the embeddings in a different way 
I made an embedding layer for the position then I concatenated position embeddings with the word embeddings 
just thought it could be a usefull way to encode the positions 

had to reshape the output of the transformer layer to get the prediction
"""
class TextTransformer(nn.Module):
  def __init__(self):
    super(TextTransformer,self).__init__()
    self.wordEmbeddings = nn.Embedding(len(TEXT.vocab),140)
    self.positionEmbeddings = nn.Embedding(140,20)
    self.transformerLayer = nn.TransformerEncoderLayer(160,8) 
    self.linear1 = nn.Linear(160,  64)
    self.linear2 = nn.Linear(64,  1)
    self.linear3 = nn.Linear(140,  16)
    self.linear4 = nn.Linear(16,  1)
  def forward(self,x):
    positions = (torch.arange(0,140).reshape(1,140) + torch.zeros(x.shape[0],140)).to(device) 
    # broadcasting the tensor of positions 
    sentence = torch.cat((self.wordEmbeddings(x.long()),self.positionEmbeddings(positions.long())),axis=2)
    attended = self.transformerLayer(sentence)
    linear1 = F.relu(self.linear1(attended))
    linear2 = F.relu(self.linear2(linear1))
    linear2 = linear2.view(-1,140) # reshaping the layer as the transformer outputs a 2d tensor (or 3d considering the batch size)
    linear3 = F.relu(self.linear3(linear2))
    out = torch.sigmoid(self.linear4(linear3))
    return out

myTransformer = TextTransformer()
myTransformer.to(device)

    


TextTransformer(
  (wordEmbeddings): Embedding(36857, 140)
  (positionEmbeddings): Embedding(140, 20)
  (transformerLayer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): Linear(in_features=160, out_features=160, bias=True)
    )
    (linear1): Linear(in_features=160, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=160, bias=True)
    (norm1): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (linear1): Linear(in_features=160, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=1, bias=True)
  (linear3): Linear(in_features=140, out_features=16, bias=True)
  (linear4): Linear(in_features=16, out_features=1, bias=True)
)

In [12]:
def calculateMetrics(ypred,ytrue):
  acc  = accuracy_score(ytrue,ypred)
  f1  = f1_score(ytrue,ypred)
  f1_average  = f1_score(ytrue,ypred,average="macro")
  return " f1 score: "+str(round(f1,3))+" f1 average: "+str(round(f1_average,3))+" accuracy: "+str(round(acc,3))
  

# Training Algorithm

In [13]:
"""
using adagrad because it assign bigger updates to less frequently updated weights 
(like words that are not used many times)

"""

optimizer = optim.Adagrad(myTransformer.parameters(),lr = 0.001)

for i in range(20):
  trainpreds = torch.tensor([])
  traintrues = torch.tensor([])
  for  batch in train_iterator:
    X = batch.comment_text
    y = batch.toxic
    myTransformer.zero_grad()
    pred = myTransformer(X).squeeze()
    trainpreds = torch.cat((trainpreds,pred.cpu().detach()))
    traintrues = torch.cat((traintrues,y.cpu().detach()))
    err = F.binary_cross_entropy(pred,y)
    err.backward()
    optimizer.step()
  err = F.binary_cross_entropy(trainpreds,traintrues)
  print("train BCE loss: ",err.item(),calculateMetrics(torch.round(trainpreds).numpy(),traintrues.numpy()))
 

  valpreds = torch.tensor([])
  valtrues = torch.tensor([])
  for batch in valid_iterator:
    X = batch.comment_text
    y = batch.toxic
    valtrues = torch.cat((valtrues,y.cpu().detach()))
    pred = myTransformer(X).squeeze().cpu().detach()
    # print(valtrues.shape)
    valpreds = torch.cat((valpreds,pred))
  err = F.binary_cross_entropy(valpreds,valtrues)
  print("validation BCE loss: ",err.item(),calculateMetrics(torch.round(valpreds).numpy(),valtrues.numpy()))
  

train BCE loss:  0.4033504128456116  f1 score: 0.587 f1 average: 0.733 accuracy: 0.814
validation BCE loss:  0.300700306892395  f1 score: 0.726 f1 average: 0.821 accuracy: 0.871
train BCE loss:  0.2577840983867645  f1 score: 0.794 f1 average: 0.86 accuracy: 0.892
validation BCE loss:  0.2441278100013733  f1 score: 0.796 f1 average: 0.864 accuracy: 0.898
train BCE loss:  0.22502966225147247  f1 score: 0.822 f1 average: 0.879 accuracy: 0.905
validation BCE loss:  0.2225685864686966  f1 score: 0.817 f1 average: 0.877 accuracy: 0.906
train BCE loss:  0.20954887568950653  f1 score: 0.838 f1 average: 0.889 accuracy: 0.913
validation BCE loss:  0.21045918762683868  f1 score: 0.828 f1 average: 0.884 accuracy: 0.91
train BCE loss:  0.20062369108200073  f1 score: 0.846 f1 average: 0.894 accuracy: 0.917
validation BCE loss:  0.20341674983501434  f1 score: 0.838 f1 average: 0.89 accuracy: 0.914
train BCE loss:  0.1946922242641449  f1 score: 0.851 f1 average: 0.898 accuracy: 0.919
validation BCE lo

# Save Algorithm

In [15]:
torch.save(myTransformer.state_dict(),"Model")
torch.save(myTransformer.state_dict, "Model.pt")
torch.save(myTransformer.state_dict, "Model.pickle")


/opt/conda/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type TextTransformer. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


# Inference on test data

In [18]:
"""
now getting the results on the test set
"""

testpreds = torch.tensor([])
testtrues = torch.tensor([])
for batch in test_iterator:
    X = batch.comment_text
    y = batch.toxic
    testtrues = torch.cat((testtrues,y.cpu().detach()))
    pred = myTransformer(X).squeeze().cpu().detach()
    # print(valtrues.shape)
    testpreds = torch.cat((testpreds,pred))
err = F.binary_cross_entropy(testpreds,testtrues)
print("test BCE loss: ",err.item(),calculateMetrics(torch.round(testpreds).numpy(),testtrues.numpy()))
  

test BCE loss:  0.17359605431556702  f1 score: 0.867 f1 average: 0.909 accuracy: 0.928


In [19]:
test["predicted"] = torch.round(testpreds).numpy()


"""
this shows that the model understands the language well 

"""

test[test.predicted==1].iloc[32:37]


,URL,Label,predicted
108,mygreenstone.ca/security/secure-code7/security...,1,1.0
114,brynncampbell.com/cache/domain/secure-domains/...,1,1.0
118,syamasahithi.com/8fh34f3,1,1.0
121,www.ppok.asia/login/en/index.htm,1,1.0
128,platinumwindowcleaning.com/wp-content/plugins/...,1,1.0
